In [4]:
import requests
from bs4 import BeautifulSoup
import sys
import pandas as pd
import numpy as np
from time import sleep
from random import randint

###  Процедура по выводу progressbar

In [5]:
def progress(count, total, suffix=''):
    bar_len = 40
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 2)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', suffix))
    sys.stdout.flush()

In [3]:
# for i in range(3564):
#     progress(i, 3563, 'обработано')

In [454]:
# START_URL = 'https://www.b2b-center.ru/market/?show=archive'
# HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '+\
#            'Chrome/86.0.4240.111 Safari/537.36',
#            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;'+\
#            'q=0.8,application/signed-exchange;v=b3;q=0.9'}
# HOST = 'https://www.b2b-center.ru'
# REC_PER_PAGE = 20

In [10]:
START_URL = 'https://m.b2b-center.ru/market/?show=archive'
HEADERS = {'user-agent': 'Mozilla/5.0 (Linux; Android 10; Redmi Note 9 Pro) AppleWebKit/537.36' + \
           '(KHTML, like Gecko) Chrome/86.0.4240.110 Mobile Safari/537.36',
           'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;'+\
           'q=0.8,application/signed-exchange;v=b3;q=0.9'}
HOST = 'https://www.b2b-center.ru'
REC_PER_PAGE = 10

### Иницилизация DataFrame

In [400]:
def df_init():
    global df_columns
    global df
    df_columns=['Id','Type','Name','Sponsor','Link','Start_date','End_date']
    df = pd.DataFrame(columns=df_columns)
    # df['Id'] = df['Id'].astype('int64')

### Функция по заполнению ДатаФрейма данными из страницы со списком закупок

In [417]:
def parse_list_page(html):
    soup = BeautifulSoup(html.text, 'html.parser')
    search_table = soup.find_all('table', class_='table table-hover table-filled search-results')

    links = search_table[0].tbody.find_all('a')
    dates = search_table[0].tbody.find_all('td', class_='nowrap')

    for i in range(0, len(links), 2):
        auc_id = links[i].text.split('\n')[0].split('№')[1].strip() # Номер торгов
        if not len(df[df['Id'] == auc_id]):
            auc_type = links[i].text.split('\n')[0].split('№')[0].strip() # Вид торгов
            auc_name = links[i].text.split('\n')[1].strip() # if len(links[i].text.split('\n')) > 1 else '11' # Название торгов
            auc_name = np.nan if len(auc_name) == 0 else auc_name
            sponsor = np.nan if len(links[i+1].text.strip()) == 0 else links[i+1].text.strip() # Наименование органзатора
            auc_link = HOST + links[i].get('href').split('#')[0] # Ссылка на страницу торгов
            start_date = dates[i].text # Колонка "Опубликовано"
            end_date = dates[i+1].text # Колонка "Актуально до"
            add_list = [auc_id, auc_type, auc_name, sponsor, auc_link, start_date, end_date]
            df.loc[len(df)] = add_list

In [24]:
%%time
for i in range(1, 100):
    curl = START_URL + '&from_archive=' + str(i*10)
    html = get_html(curl)
    print(i, end= ' ')

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 

KeyboardInterrupt: 

In [20]:
soup = BeautifulSoup(html.text, 'html.parser')
html.text

# search_table = soup.find_all('table', class_='table table-hover table-filled search-results')

# links = search_table[0].tbody.find_all('a')
# dates = search_table[0].tbody.find_all('td', class_='nowrap')
# links[39].text.strip()

'<!doctype html>\n<html lang="ru">\n<head>\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, viewport-fit=cover, shrink-to-fit=no">\n    <meta name="format-detection" content="telephone=no">\n                        <title>Закупки товаров и услуг на сайте электронной торговой площадки B2B-Center</title>\n                            <meta name="description" content="Поиск тендеров, закупок и торгов по классификатору ОКПД2. Найдите для своей компании подходящую закупку с помощью удобной системы поиска на торговой площадке B2B-Center.">\n                            <meta name="keywords" content="электронная торговая площадка, электронные торги, продажа, закупка, товары, услуги, электронная подпись, B2B, тендеры, аукционы, запросы цен, квалификационные отборы, объявления о продаже.">\n            \n    <link rel="icon" type="image/x-icon" href="/images/projects/b2bmobile/favicons/favicon.ico">\n    <link rel="icon" type="i

### Функция возвращает результат HTML запроса

In [7]:
def get_html(url, params=None):
    r = requests.get(url, headers=HEADERS, params=params)
    return r

### Функция возвращает элемент url адреса from для последней страницы

In [8]:
def get_last_page(html):
    soup = BeautifulSoup(html.text, 'html.parser')
    search_pagination = soup.find_all('li', class_='pagi-item')
    return int(search_pagination[-1].a.get('href').split('from_archive=')[1])

In [456]:
%%time
df_init()
html = get_html(START_URL)

last_page = get_last_page(html)
last_page = 980
parse_list_page(html)

for i in range(REC_PER_PAGE, last_page + REC_PER_PAGE, REC_PER_PAGE):
    curl = START_URL + '&from_archive=' + str(i)
    html = get_html(curl)
    parse_list_page(html)
    progress(i, last_page)
    sleep(randint(10, 20) / 10)

IndexError: list index out of range

In [9]:
html

NameError: name 'html' is not defined

In [467]:
# df.info()
#df['Sponsor'].value_counts()
#df.to_excel('export/first_960.xls')
#df.to_pickle('export/first_960.pkl')

In [479]:
df.loc[df['Type'].str.contains('Закры')]

,Id,Type,Name,Sponsor,Link,Start_date,End_date
1,2493647,Закрытый запрос предложений,NaN,"ООО ""Жефко""",https://www.b2b-center.ru/market/ki-24674-1-da...,30.10.2020 10:06,30.10.2020 11:15
2,2493618,Закрытый запрос предложений,Услуги по перевозке готовой продукции на 31.10,АО «Кордиант»,https://www.b2b-center.ru/market/uslugi-po-per...,30.10.2020 09:50,30.10.2020 11:01
3,2493590,Закрытый запрос предложений,Услуги по перевозке готовой продукции,АО «Кордиант»,https://www.b2b-center.ru/market/uslugi-po-per...,30.10.2020 09:47,30.10.2020 10:43
4,2493564,Закрытый запрос предложений,NaN,"ООО ""Жефко""",https://www.b2b-center.ru/market/ki-24672-1-30...,30.10.2020 09:18,30.10.2020 11:00
5,2493556,Закрытый запрос предложений,NaN,"ООО ""Жефко""",https://www.b2b-center.ru/market/zapros-stavki...,30.10.2020 09:16,30.10.2020 11:00
...,...,...,...,...,...,...,...
950,2488274,Закрытый запрос предложений,NaN,"ЗАО ""ТОРГОВЫЙ ДОМ ""ПЕРЕКРЕСТОК""",https://www.b2b-center.ru/market/vybor-postavs...,23.10.2020 17:21,28.10.2020 12:00
952,2488287,Закрытый запрос предложений,NaN,"ООО ""Велесстрой""",https://www.b2b-center.ru/market/log02-333mva/...,23.10.2020 17:13,26.10.2020 11:00
954,2488251,Закрытый запрос предложений,NaN,"ООО УК ""МЕТАЛЛОИНВЕСТ""",https://www.b2b-center.ru/market/kolodka-tormo...,23.10.2020 17:06,29.10.2020 10:00
956,2486373,Закрытый запрос предложений,«Поставка арматурного металлопроката автотранс...,"ООО ""ГК ФСК""",https://www.b2b-center.ru/market/postavka-arma...,23.10.2020 17:03,26.10.2020 19:55
